<a href="https://colab.research.google.com/github/jmyth742/EEG-BCI/blob/master/BetterDL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [209]:
import numpy as np
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim


##### START OF ADDITION OF MY CODE

import numpy as np
import os.path
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import io
from googleapiclient.http import MediaIoBaseDownload

from scipy.io import loadmat
from pathlib import Path
import matplotlib.pyplot as plt
import scipy.io as sio
import sys

## PyTorch 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch import Tensor
import torch.optim as optim
import torchvision

import math #for calculus

## END OF IMPORTS


ImportError: ignored

In [0]:

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#file_id = '1FrXb6rTyqpE5SmNtP8HhygDmFf9Lw896'
file_id = '1al08tF3j-Z2-fowjPPNz1SrFgLVqywXG' ##1.mat
#https://drive.google.com/open?id=1TqewoCjjRXZpEIxL_23ZQ8MP4L0RofZx
file_id = '1TqewoCjjRXZpEIxL_23ZQ8MP4L0RofZx'
#https://drive.google.com/open?id=1al08tF3j-Z2-fowjPPNz1SrFgLVqywXG
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('./train3.mat')
#mat = loadmat('train3.mat')
mat = sio.loadmat('train3.mat', squeeze_me=True, struct_as_record=False)
data=[]

In [211]:
      
## in class      
device='cuda:0'
learning_rate=0.01
weight_decay=0.000001 
momentum=0.9
batch_size=200

o=mat['o']

data=mat['o'].data
#print(data[200][21])
labels=mat['o'].marker

print(labels.size)
print(labels.shape)
np.set_printoptions(threshold=sys.maxsize)
#print(labels)
pos=np.zeros((22,3), dtype=int)

pos[0]=-2,10,0
pos[1]=2,10,0
pos[3]=-4,7,3
pos[4]=4,7,3
pos[5]=-5,0,4
pos[6]=5,0,4
pos[7]=-4,-7,3
pos[8]=4,-7,3
pos[9]=-2,-10,0
pos[10]=2,-10,0
pos[11]=-7,0,-5
pos[12]=7,0,5
pos[13]=-6,6,0
pos[14]=6,6,0
pos[15]=-7,0,0
pos[16]=7,0,0
pos[17]=-6,-6,0
pos[18]=6,-6,0
pos[19]=0,7,4
pos[20]=0,0,6
pos[21]=0,-7,4

def make_3d_point(x,y,z,r,theta,phi):
  return [
      x + int(r * math.cos(theta)*math.sin(phi)),
      y + int(r * math.sin(theta)*math.sin(phi)),
      z + int(r * math.cos(phi))
         ]

def make_3d_image(image3d_data):
  
  #creating the matrix
  matrix = np.zeros([16,21,4])
  
  #looping through data
  for point in image3d_data:
    # creating the indice by shifting the coordinate system from the center to left corner
    newpoint = [int(point[0]) + 8,int(point[1]) + 10,int(point[2]) + 1]
    
    #the maximum value
    cap = 500 + 300 #500 based on the data, 200 because of the shift
    
    #for scaling the values between 0 and 255
    scale = 255/cap
    
    if newpoint[0] >= matrix.shape[0] or newpoint[1] >= matrix.shape[1] or newpoint[2] >= matrix.shape[2] or newpoint[0] < 0 or newpoint[1] < 0 or newpoint[2] < 0:
      continue
    
    #assigning the value
    if int(point[3]) >= 0 and int(point[3]) <= cap:
      matrix[newpoint[0],newpoint[1],newpoint[2]] = int(point[3]*scale)
    elif int(point[3]) > cap:
      matrix[newpoint[0],newpoint[1],newpoint[2]] = int(cap*scale)
    else:
      matrix[newpoint[0],newpoint[1],newpoint[2]] = 0
    
    #creating variables for creating a virtual sphere around poi
    rho = [1,2,3] # distance (in pixels) that we are interpolating from poi
    theta = 0 # angle in radians [0,2pi]
    phi = 0 #angle in radians    [0, pi]
    
    #creating new interpolated points
    for r in rho:
      for i in range(0,360,45):
        for j in range(0,180,45):
          p = make_3d_point(newpoint[0],newpoint[1],newpoint[2],r,math.radians(i),math.radians(j))
          
          #checking if its in bounds
          if p[0] < matrix.shape[0] and p[1] < matrix.shape[1] and p[2] < matrix.shape[2] and p[0] >= 0 and p[1] >= 0 and p[2] >= 0:
            #assigning the value to the matrix, scaled down by distance
            if int(point[3]) >= 0 and int(point[3]) <= cap:
              if matrix[p[0],p[1],p[2]] == 0:
                matrix[newpoint[0],newpoint[1],newpoint[2]] = int(point[3]/r*scale)
            elif int(point[3]) > cap:
              if matrix[p[0],p[1],p[2]] == 0:
                matrix[newpoint[0],newpoint[1],newpoint[2]] = int(cap/r*scale)
            else:
              if matrix[p[0],p[1],p[2]] == 0:
                matrix[newpoint[0],newpoint[1],newpoint[2]] = 0
            
  return matrix


start = 250940
end   = 260940
print(end-start)
y = np.zeros([10000])
video_stream = np.zeros([end-start,16,21,4])
for i in range(0,10000,1):
  y[i] =labels[start+i]

for t in range(start,end,1):
  v = np.add(data[t,:],300).reshape([22,1])
 
  #create x,y,z,value for timestep t
  data3d = np.hstack((pos,v))
  
  #make image - with interpolated data for timestep t
  result = make_3d_image(data3d)
  
  #add image to video stream
  video_stream[t-start] = result
  
  
maxValue = np.amax(video_stream)
minValue = np.amin(video_stream)
print(y.size)

#print(y)
#print(y.shape)

667000
(667000,)
10000
10000


In [0]:
class EEGNet(nn.Module):
    def __init__(self):
        super(EEGNet, self).__init__()
        self.T = 120
        
        # Layer 1
        self.conv1 = nn.Conv2d(16, 12,(4,3), 2, padding = 1)
        #self.conv1 = nn.Conv3d(16, 1, ,7,, 1, padding = 0)
        #self.conv1 = nn.Conv2d(1, 16, (1, 64), padding = 0)
        #self.batchnorm1 = nn.BatchNorm2d(16, False)
        self.batchnorm1 = nn.BatchNorm2d(12, False)
        
        # Layer 2
        #self.padding1 = nn.ZeroPad2d((16, 17, 0, 1)) #og
        #self.conv2 = nn.Conv2d(16, 16, (2, 2))
        self.conv2 = nn.Conv2d(12, 8, (2, 2))
        #self.conv2 = nn.Conv2d(1, 4, (2, 32))#og
        self.batchnorm2 = nn.BatchNorm2d(8, False)
        self.pooling2 = nn.MaxPool2d(4, 4)
        #self.pooling2 = nn.AdaptiveAvgPool2d(2) 
        #self.pooling2 = nn.MaxPool2d(2, 4)#og
        
        # Layer 3
        #self.padding2 = nn.ZeroPad2d((2, 1, 4, 3))#og
        self.conv3 = nn.Conv2d(8, 4, (1, 1))
        #self.conv3 = nn.Conv2d(4, 4, (8, 4))#og
        self.batchnorm3 = nn.BatchNorm2d(4, False)
        #self.batchnorm3 = nn.BatchNorm2d(4, False)#og
        self.pooling3 = nn.MaxPool2d((1, 1))
        #self.pooling3 = nn.AdaptiveAvgPool2d(1) 
        #self.pooling3 = nn.MaxPool2d((2, 4))#og
        
        # FC Layer
        # NOTE: This dimension will depend on the number of timestamps per sample in your data.
        # I have 120 timepoints. 
        #self.fc1 = nn.Linear(1,10000,1)
        self.fc1 = nn.Linear(4*2*1,1)
        # self.fc1 = nn.Linear(4*2*7, 1)

    def forward(self, x):
        # Layer 1
        x = F.elu(self.conv1(x))
        #print("after elu",x.shape)
        x = self.batchnorm1(x)
        #print("after batch",x.shape)
        x = F.dropout(x, 0.25)
        #print("after dropout",x.shape)
        #x = x.permute(0, -4, 1, 2)
        
        # Layer 2
        #x = self.padding1(x)#og
        x = F.elu(self.conv2(x))
        #print("after conv2",x.shape)
        #print("after padding l2",x.shape)
        x = self.batchnorm2(x)
        #print("after batch2",x.shape)
        x = F.dropout(x, 0.25)
        #print("after dropout2",x.shape)
        x = self.pooling2(x)
        #print("after pooling2",x.shape)
        
        # Layer 3
        #x = self.padding2(x)#og
        x = F.elu(self.conv3(x))
        #print("after conv3",x.shape)
        x = self.batchnorm3(x)
        #print("after batch3",x.shape)
        x = F.dropout(x, 0.25)
        #print("after droup3",x.shape)
        x = self.pooling3(x)
        #print("after pool3",x.shape) 

        # FC Layer
        #x = x.view(-1, 4*2*7)#og
        #x = x.view(-1, 10000*1*1*1)
        x = x.view(-1,4*2*1)
        #print("x view shape",x.shape)
        #x.view(x.size(0),-1,10000*1*1*1)
        x = F.sigmoid(self.fc1(x))
        #print("after sigmoid",x.shape) 
        return x    
      
      
def evaluate(model, X, Y, params = ["acc"]):
    results = []
    batch_size = 10000
    
    predicted = []
    for i in range(len(X)//batch_size):
        s = i*batch_size
        e = i*batch_size+batch_size
        
        inputs = Variable(torch.from_numpy(X[s:e]))
        pred = model(inputs.float())
        
        predicted.append(pred.data.cpu().numpy())
        
    inputs = Variable(torch.from_numpy(X))

    predicted = model(inputs.float())

    predicted = predicted.data.cpu().numpy()

    
    for param in params:
        if param == 'acc':
            #print("acc causing issue")
            #print("y shape is ", Y.shape)
            #print("predicted shape is ",np.round(predicted).shape)
            results.append(accuracy_score(Y, np.round(predicted)))
        #if param == "auc":
            #print("auc causing issue")
            #results.append(roc += roc_auc_score(Y, np.round(predicted)))
        #if param == "recall":
            #print("recall causing issue")
            #results.append(recall_score(Y, np.round(predicted)))
        #if param == "precision":
            #print("precision causing issue")
            #results.append(precision_score(Y, np.round(predicted)))
        #if param == "fmeasure":
            #print("fmeasure causing issue")
            #precision = precision_score(Y, np.round(predicted))
            #recall = recall_score(Y, np.round(predicted))
            #results.append(2*precision*recall/ (precision+recall))
    return results

      
#print(maxValue)
#print(minValue)

   
#print(video_stream.shape)
#print(img)

## END OF ADDITION OF MY CODE
      
            
net = EEGNet().float()
#print(net.forward(Variable(torch.Tensor(np.random.rand(1, 1, 120, 64)))))
net.forward(Variable(torch.FloatTensor(video_stream)))
#criterion = nn.BCELoss()
criterion =  nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())

#print("video stream size ", video_stream.size)
#print("video stream shape ", video_stream.shape)
video_stream.astype(float)

#video stream size  13440000
#video stream shape  (10000, 16, 21, 4)

#X_train = np.random.rand(100, 1, 120, 64).astype('float32') # np.random.rand generates between [0, 1)
X_train = video_stream.astype('float32') # our generated image
#y_train = np.round(np.random.rand(100).astype('float32')) # binary data, so we round it to 0 or 1.
#y = label_binarize(y, classes=[0, 1, 2, 3])
#n_classes = y.shape[1]
y_train = y.astype('float32')
#y_train = labels.astype('float32')

X_val = video_stream.astype('float32')
#X_val = np.random.rand(200, 16, 21, 4).astype('float32')
#y_val = np.round(np.random.rand(100).astype('float32'))#og
y_val = y.astype('float32')

X_test = video_stream.astype('float32')
#X_test = np.random.rand(200, 16, 21, 4).astype('float32')
#y_test = np.round(np.random.rand(100).astype('float32'))
y_test =  y.astype('float32')


#print(X_train.shape)
#print("video stream shape and size")
#print(video_stream.shape)
#print(video_stream.size)
#print("y train shape and size")
#
print(y_train.size)
#print(y_train.shape)
#print()

batch_size = 20000

for epoch in range(100000):  # loop over the dataset multiple times
    print("\nEpoch ", epoch)
    
    running_loss = 0.0
    for i in range(len(X_train)//batch_size-1):
        s = i*batch_size
        e = i*batch_size+batch_size
        
        inputs = torch.from_numpy(X_train[s:e])
        labels = torch.FloatTensor(np.array([y_train[s:e]]))#.T*1.0
        
        # wrap them in Variable
        inputs, labels = Variable(inputs), Variable(labels)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs.float())
        loss = criterion(outputs, labels)
        loss.backward()
        
        
        optimizer.step()
        
        running_loss += loss.data
        print(running_loss)
    
    # Validation accuracy
    params = ["acc", "auc", "fmeasure"]
    print(params)
    print("Training Loss ", running_loss)
    print("Train - ", evaluate(net, X_train, y_train, params))
    print("Validation - ", evaluate(net, X_val, y_val, params))
    print("Test - ", evaluate(net, X_test, y_test, params))


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1386: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


10000

Epoch  0
['acc', 'auc', 'fmeasure']
Training Loss  0.0
Train -  [0.2721]
Validation -  [0.2788]
Test -  [0.2838]

Epoch  1
['acc', 'auc', 'fmeasure']
Training Loss  0.0
Train -  [0.2758]
Validation -  [0.284]
Test -  [0.2836]

Epoch  2
['acc', 'auc', 'fmeasure']
Training Loss  0.0
Train -  [0.2805]
Validation -  [0.2796]
Test -  [0.2826]

Epoch  3
['acc', 'auc', 'fmeasure']
Training Loss  0.0
Train -  [0.2776]
Validation -  [0.2786]
Test -  [0.2809]

Epoch  4
['acc', 'auc', 'fmeasure']
Training Loss  0.0
Train -  [0.2758]
Validation -  [0.2824]
Test -  [0.2832]

Epoch  5
['acc', 'auc', 'fmeasure']
Training Loss  0.0
Train -  [0.2767]
Validation -  [0.282]
Test -  [0.2748]

Epoch  6
['acc', 'auc', 'fmeasure']
Training Loss  0.0
Train -  [0.2749]
Validation -  [0.282]
Test -  [0.2785]

Epoch  7
['acc', 'auc', 'fmeasure']
Training Loss  0.0
Train -  [0.2793]
Validation -  [0.2796]
Test -  [0.2775]

Epoch  8
['acc', 'auc', 'fmeasure']
Training Loss  0.0
Train -  [0.2779]
Validation 